# Tutorial


This tutorial runs you through the process of running inferences for a deployments in Costa Rica.

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
# set the wd
os.chdir(os.path.expanduser('~/amber-inferences'))

In [ ]:
# Install the package if required
%pip install -e .

In [ ]:
from PIL import ImageFont, ImageDraw, Image

import boto3
import pandas as pd
from tqdm import tqdm

import amber_inferences
from amber_inferences.utils.config import load_credentials
from amber_inferences.utils.api_utils import get_buckets, deployments_summary, get_deployments
from amber_inferences.utils.custom_models import *
from amber_inferences.utils.inference_scripts import *

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display
from IPython.display import Markdown as md

import torch

# Explore the Data on the Object Store

In [ ]:
# Create an instance for the object store
aws_credentials = load_credentials('./credentials.json')
session = boto3.Session(
        aws_access_key_id=aws_credentials["AWS_ACCESS_KEY_ID"],
        aws_secret_access_key=aws_credentials["AWS_SECRET_ACCESS_KEY"],
        region_name=aws_credentials["AWS_REGION"],
    )
s3_client = session.client("s3", endpoint_url=aws_credentials["AWS_URL_ENDPOINT"])

**🚨 Note: this feature has been taken down by Posit so the app is not currently available 🚨**

Look at the deployments available on the object store:

In [ ]:
# all_deployments = get_deployments(aws_credentials['UKCEH_username'], aws_credentials['UKCEH_password'])

List the buckets/countries:

In [ ]:
# all_deployments = pd.DataFrame(all_deployments)
# all_deployments[all_deployments['status'] == 'active']

Let's pick one, cri (Costa Rica) and check out the data attached. 

In [ ]:
# cr_deployments = deployments_summary(
#     aws_credentials,
#     subset_countries=["Costa Rica"],
#     subset_deployments=["dep000035", "dep000036"],
#     include_image_count=False
# )

To get the files for a given deployment(s):

In [ ]:
# cr_deployments

# Log the image keys

In [ ]:
from amber_inferences.utils.key_utils import save_keys

In [ ]:
# takes some time to commenting out to save time
# save_keys(
#     s3_client,
#     bucket="cri",
#     deployment_id="dep000035",
#     output_file="./examples/example_keys/dep000035_keys.json",
#     subdir="snapshot_images"
# )

In [ ]:
# Look at the keys
!head ./examples/example_keys/interesting_timelapse.json

# Download the images

In [ ]:
import json
from amber_inferences.utils.inference_scripts import download_image_from_key

In [ ]:
# read the first image in the keys file and open
with open('./examples/example_keys/interesting_timelapse.json') as f:
    keys = json.load(f)

In [ ]:
os.makedirs('./examples/images/dep000035/interesting_timelapse/raw/', exist_ok=True)

for i in range(len(keys)):
    download_image_from_key(s3_client, keys[i], 'cri', './examples/images/dep000035/interesting_timelapse/raw/')

In [ ]:
image_paths = os.listdir('./examples/images/dep000035/interesting_timelapse/raw/')
image_paths = [os.path.join('./examples/images/dep000035/interesting_timelapse/raw/', x) for x in image_paths]

image_paths = [x for x in image_paths if x.endswith('.jpg')]

len(image_paths)

In [ ]:
# # Open the images in a 2x5 grid
# fig, axs = plt.subplots(5, 6, figsize=(20, 20))
# axs = axs.ravel()

# for i, img_path in enumerate(image_paths):
#     if os.path.exists( img_path):  # Ensure the file exists
#         img = mpimg.imread(img_path)
#         axs[i].imshow(img)
#         axs[i].axis("off")  # Hide axes for better visualization
#         axs[i].set_title(f"Image {i+1}")
#     else:
#         axs[i].axis("off")
#         axs[i].set_title("Missing Image")

# plt.tight_layout()
# plt.show()

In [ ]:
# Open images and convert to a sequence
images = [Image.open(img) for img in image_paths]

# Save as GIF
os.makedirs('./examples/images/dep000035/interesting_timelapse/gifs', exist_ok=True)
gif_path = "./examples/images/dep000035/interesting_timelapse/gifs/raw_images.gif"
images[0].save(gif_path, save_all=True, append_images=images[1:], duration=500, loop=0)

del images

In [ ]:
md("![mothGif](" + os.path.abspath(gif_path) + " 'moth')")

# Perform Object Detection on the Images

In [ ]:
# set the torch device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

In [ ]:
models_load = load_models(
    device=device,
    localisation_model_path='./models/v1_localizmodel_2021-08-17-12-06.pt',
    binary_model_path='./models/moth-nonmoth-effv2b3_20220506_061527_30.pth',
    order_model_path='./models/dhc_best_128.pth',
    order_threshold_path='./models/thresholdsTestTrain.csv',
    species_model_path='./models/turing-costarica_v03_resnet50_2024-06-04-16-17_state.pt',
    species_labels='./models/03_costarica_data_category_map.json'
)

In [ ]:
def image_annotation(image_path, img=None, boxes={}, scale=False, default_colour='grey'):
    if img is None:
        img = Image.open(image_path)

    draw = ImageDraw.Draw(img)

    for box in boxes:
        x0 = float(box['x_min'])
        y0 = float(box['y_min'])
        x1 = float(box['x_max'])
        y1 = float(box['y_max'])
        if scale:
            og_width, og_height = img.size
            x0 = x0/300*og_width
            y0 = y0/300*og_height
            x1 = x1/300*og_width
            y1 = y1/300*og_height
        if 'ann_col' not in box.keys():
            box['ann_col'] = default_colour
        if 'label' not in box.keys():
            box['label'] = ''

        draw.rectangle([x0, y0, x1, y1], outline=box['ann_col'], width=3)
        draw.text((x0, y0), box['label'], fill=box['ann_col'],
                  font=ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", size=50) )

    return img

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image

# Load a pretrained feature extractor (e.g., ResNet50 without classifier)
resnet = models.resnet50(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(resnet.children())[:-1])  # remove final FC
feature_extractor.eval()

# Transform for input crops
embedding_transform = transforms.Compose(
    [
        transforms.Resize((300, 300)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ]
)

def extract_embedding(crop):
    """Crop: a PIL image."""
    input_tensor = embedding_transform(crop).unsqueeze(0)  # Add batch dim
    with torch.no_grad():
        embedding = feature_extractor(input_tensor).squeeze().numpy()  # Shape (2048,)
    return embedding


In [ ]:
all_crops_loc = []
embeddings_list = {}

os.makedirs('./examples/images/dep000035/interesting_timelapse/annotated_boxes/', exist_ok=True)
os.makedirs('./examples/images/crops/interesting_timelapse/', exist_ok=True)

for i, img_path in tqdm(enumerate(image_paths[0:5])):
    crops = crop_image_only(
        image_path=img_path,
        bucket_name="cri",
        localisation_model=models_load['localisation_model'],
        proc_device=device,
        csv_file="./examples/interesting_timelapse_crops.csv",
        save_crops=True,
        box_threshold=0.95,
        crop_dir="./examples/images/crops/interesting_timelapse",
        job_name=None,
    )

    boxes = []
    all_crops_loc.append(crops)
    image = Image.open(img_path).convert("RGB")
    embeddings_list[img_path] = {}

    for j, row in crops.iterrows():
        boxes.append({
            'x_min': row['x_min'],
            'y_min': row['y_min'],
            'x_max': row['x_max'],
            'y_max': row['y_max'],
            'label': row['crop_status'],
            'ann_col': 'grey'
        })


        # Crop original image and extract embedding
        crop = image.crop((row['x_min'], row['y_min'], row['x_max'], row['y_max']))
        embedding = extract_embedding(crop)
        embeddings_list[img_path][row['crop_status']] = {
            'embedding': embedding,
            'file': os.path.basename(row['image_path']),
            'crop': row['crop_status'],
            'crop_image': crop,
            'box': {'xmin':row['x_min'], 'ymin':row['y_min'], 'xmax':row['x_max'], 'ymax':row['y_max']}
        }

    del crops
    img = image_annotation(img_path, boxes=boxes)

    # save the image
    img.save(f'./examples/images/dep000035/interesting_timelapse/annotated_boxes/{os.path.basename(img_path)}')

In [ ]:
all_crops_loc = pd.concat(all_crops_loc)
all_crops_loc = all_crops_loc.reset_index(drop=True)

In [ ]:
# fig, axs = plt.subplots(8, 4, figsize=(20, 30))
# axs = axs.ravel()

# for i, img in enumerate(imgs):
#     axs[i].imshow(img)
#     axs[i].axis('off')
#     axs[i].set_title(f"Image {i+1}")

# plt.tight_layout()
# plt.show()

Save to gif

In [ ]:
# Open images and convert to a sequence
image_paths = os.listdir('./examples/images/dep000035/interesting_timelapse/annotated_boxes/')
image_paths = [os.path.join('./examples/images/dep000035/interesting_timelapse/annotated_boxes/', x) for x in image_paths]
images = [Image.open(img) for img in image_paths]

# Save as GIF
gif_path = "./examples/images/dep000035/interesting_timelapse/gifs/object_detection_images.gif"
images[0].save(gif_path, save_all=True, append_images=images[1:], duration=500, loop=0)

del images

In [ ]:
md("![mothGif](" + os.path.abspath(gif_path) + " 'moth')")

# Flatbug

⚠️ This section is only advised if cuda is available

In [ ]:
models = load_models(
    device=device,
    localisation_model_path='./models/flat_bug_M.pt',
    binary_model_path='./models/moth-nonmoth-effv2b3_20220506_061527_30.pth',
    order_model_path='./models/dhc_best_128.pth',
    order_threshold_path='./models/thresholdsTestTrain.csv',
    species_model_path='./models/turing-costarica_v03_resnet50_2024-06-04-16-17_state.pt',
    species_labels='./models/03_costarica_data_category_map.json'
)

In [ ]:
torch.cuda.is_available()

In [ ]:
if torch.cuda.is_available():
    os.makedirs('./examples/images/dep000035/interesting_timelapse/annotated_boxes_flatbug/', exist_ok=True)
    os.makedirs('./examples/images/crops/interesting_timelapse_flatbug/', exist_ok=True)

    all_crops = []

    for i, img_path in tqdm(enumerate(image_paths)):
        crops = crop_image_only(
            image_path=img_path,
            bucket_name="cri",
            localisation_model=models['localisation_model'],
            proc_device=device,
            csv_file="./examples/interesting_timelapse_flatbug.csv",
            save_crops=True,
            box_threshold=0,
            crop_dir="./examples/images/crops/interesting_timelapse_flatbug",
            job_name=None,
        )
        crops = crops.loc[crops['crop_status'] != 'NO DETECTIONS FOR IMAGE',]

        all_crops = all_crops + [crops]
        if crops.shape[0] > 0:
            boxes = []
            for j, row in crops.iterrows():
                boxes.append({
                    'x_min': row['x_min'],
                    'y_min': row['y_min'],
                    'x_max': row['x_max'],
                    'y_max': row['y_max'],
                    'label': '',
                    'ann_col': 'grey'
                })
            del crops
            img = image_annotation(img_path, boxes=boxes, scale=False)

            # save the image
            img.save(f'./examples/images/dep000035/interesting_timelapse/annotated_boxes_flatbug/{os.path.basename(img_path)}')

In [ ]:
# fig, axs = plt.subplots(8, 4, figsize=(20, 30))
# axs = axs.ravel()

# for i, img in enumerate(imgs):
#     axs[i].imshow(img)
#     axs[i].axis('off')
#     axs[i].set_title(f"Image {i+1}")

# plt.tight_layout()
# plt.show()

In [ ]:
if torch.cuda.is_available():
    # Open images and convert to a sequence
    image_paths = os.listdir('./examples/images/dep000035/interesting_timelapse/annotated_boxes_flatbug/')
    image_paths = [os.path.join('./examples/images/dep000035/interesting_timelapse/annotated_boxes_flatbug/', x) for x in image_paths]
    images = [Image.open(img) for img in image_paths]

    # Save as GIF
    gif_path = "./examples/images/dep000035/interesting_timelapse/gifs/flatbug_detection_images.gif"
    images[0].save(gif_path, save_all=True, append_images=images[1:], duration=500, loop=0)

    # display(Image.open(gif_path))
    del images

    all_crops = pd.concat(all_crops)
    all_crops = all_crops.reset_index(drop=True)

In [ ]:
if torch.cuda.is_available():
    md("![mothGif](" + os.path.abspath(gif_path) + " 'moth')")

# Binary Classifier

In [ ]:
os.makedirs('./examples/images/dep000035/interesting_timelapse/annotated_boxes_binary/', exist_ok=True)

transform_species = transforms.Compose(
        [
            transforms.Resize((300, 300)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ]
    )

In [ ]:
image_paths = os.listdir('./examples/images/dep000035/interesting_timelapse/raw/')
image_paths = [os.path.abspath(os.path.join('./examples/images/dep000035/interesting_timelapse/raw/', x)) for x in image_paths]

In [ ]:
for i, image_path in tqdm(enumerate(image_paths))):
    imge = Image.open(image_path).convert("RGB")
    original_image = imge.copy()
    original_width, original_height = imge.size

    crops_df = all_crops.loc[all_crops['image_path'] == image_path, ]
    crops_df = crops_df.loc[crops_df['crop_status'] != 'NO DETECTIONS FOR IMAGE',]

    if crops_df.shape[0] > 0:
        boxes = []
        for j, row in crops_df.iterrows():
            ann_col='red'
            cropped_image = original_image.crop((row['x_min'], row['y_min'], row['x_max'], row['y_max']))
            cropped_tensor = transform_species(cropped_image).unsqueeze(0).to(device)
            binary_prediction = classify_box(cropped_tensor, models['classification_model'])
            if binary_prediction[0] == 'moth':
                ann_col='green'

            boxes.append({
                'x_min': row['x_min'],
                'y_min': row['y_min'],
                'x_max': row['x_max'],
                'y_max': row['y_max'],
                'label': binary_prediction[0],
                'ann_col': ann_col
            })

        im = image_annotation(image_path, boxes=boxes, scale=False)
        im.save(f'./examples/images/dep000035/interesting_timelapse/annotated_boxes_binary/{os.path.basename(image_path)}')

In [ ]:
# Open images and convert to a sequence
image_paths = os.listdir('./examples/images/dep000035/interesting_timelapse/annotated_boxes_binary/')
image_paths = [os.path.join('./examples/images/dep000035/interesting_timelapse/annotated_boxes_binary/', x) for x in image_paths]
images = [Image.open(img) for img in image_paths]

# Save as GIF
gif_path = "./examples/images/dep000035/interesting_timelapse/gifs/binary_images.gif"
images[0].save(gif_path, save_all=True, append_images=images[1:], duration=500, loop=0)

# display(Image.open(gif_path))
del images

In [ ]:
md("![mothGif](" + os.path.abspath(gif_path) + " 'moth')")

# Order Classifier

In [ ]:
all_crops['image_path'][0]

In [ ]:
image_paths = os.listdir('./examples/images/dep000035/interesting_timelapse/raw/')
image_paths = [os.path.abspath(os.path.join('./examples/images/dep000035/interesting_timelapse/raw/', x)) for x in image_paths]
image_paths[0]

In [ ]:
os.makedirs('./examples/images/dep000035/interesting_timelapse/annotated_boxes_order/', exist_ok=True)

imgs = []
for i, image_path in enumerate(image_paths):
    imge = Image.open(image_path).convert("RGB")
    original_image = imge.copy()
    original_width, original_height = imge.size

    crops_df = all_crops.loc[all_crops['image_path'] == image_path, ]
    crops_df = crops_df.loc[crops_df['crop_status'] != 'NO DETECTIONS FOR IMAGE',]

    if crops_df.shape[0] > 0:
        boxes = []
        for j, row in crops_df.iterrows():
            ann_col = 'red'
            cropped_image = original_image.crop((row['x_min'], row['y_min'], row['x_max'], row['y_max']))
            cropped_tensor = transform_species(cropped_image).unsqueeze(0).to(torch.device('cuda:0'))
            order_prediction = classify_order(
                cropped_tensor,
                models['order_model'],
                models['order_model_labels'],
                models['order_model_thresholds']
            )
            if 'Lepidoptera' in order_prediction[0]:
                ann_col = 'green'

            boxes.append({
                'x_min': row['x_min'],
                'y_min': row['y_min'],
                'x_max': row['x_max'],
                'y_max': row['y_max'],
                'label': order_prediction[0],
                'ann_col': ann_col
            })

        im = image_annotation(image_path, boxes=boxes, scale=False)
        im.save(f'./examples/images/dep000035/interesting_timelapse/annotated_boxes_order/{os.path.basename(image_path)}')

        imgs = imgs + [im]


In [ ]:
# Open images and convert to a sequence
image_paths = os.listdir('./examples/images/dep000035/interesting_timelapse/annotated_boxes_order/')
image_paths = [os.path.join('./examples/images/dep000035/interesting_timelapse/annotated_boxes_order/', x) for x in image_paths]
images = [Image.open(img) for img in image_paths]

# Save as GIF
gif_path = "./examples/images/dep000035/interesting_timelapse/gifs/order_images.gif"
images[0].save(gif_path, save_all=True, append_images=images[1:], duration=500, loop=0)

# display(Image.open(gif_path))
del images

# Species Classifier

In [ ]:
image_paths = os.listdir('./examples/images/dep000035/interesting_timelapse/raw/')
image_paths = [os.path.abspath(os.path.join('./examples/images/dep000035/interesting_timelapse/raw/', x)) for x in image_paths]

# all_crops['image_path'] = all_crops['image_path'].replace('annotated_boxes', 'raw')

print(image_paths[0])
print(all_crops['image_path'][0])

In [ ]:
os.makedirs('./examples/images/dep000035/interesting_timelapse/annotated_boxes_species/', exist_ok=True)

imgs = []
for i, image_path in enumerate(image_paths):
    imge = Image.open(image_path).convert("RGB")
    original_image = imge.copy()
    original_width, original_height = imge.size

    crops_df = all_crops.loc[all_crops['image_path'] == image_path, ]
    crops_df = crops_df.loc[crops_df['crop_status'] != 'NO DETECTIONS FOR IMAGE',]

    if crops_df.shape[0] > 0:
        boxes = []
        for j, row in crops_df.iterrows():
            cropped_image = original_image.crop((row['x_min'], row['y_min'], row['x_max'], row['y_max']))
            cropped_tensor = transform_species(cropped_image).unsqueeze(0).to(torch.device('cuda:0'))
            order_prediction = classify_order(
                cropped_tensor,
                models['order_model'],
                models['order_model_labels'],
                models['order_model_thresholds']
            )
            label = ""
            ann_col = 'red'

            if 'Lepidoptera' in order_prediction[0]:
                species_names, species_confidences = classify_species(
                    cropped_tensor,
                    models['species_model'],
                    models['species_model_labels'],
                    5
                )
                label = f"{species_names[0]}, {'%.2f'.format(species_confidences[0]*100)}%"
                ann_col='green'

            boxes.append({
                'x_min': row['x_min'],
                'y_min': row['y_min'],
                'x_max': row['x_max'],
                'y_max': row['y_max'],
                'label': label,
                'ann_col': ann_col
            })

        im = image_annotation(image_path, boxes=boxes, scale=False)
        im.save(f'./examples/images/dep000035/interesting_timelapse/annotated_boxes_species/{os.path.basename(image_path)}')

In [ ]:
image_paths = os.listdir('./examples/images/dep000035/interesting_timelapse/annotated_boxes_species/')
image_paths = [os.path.join('./examples/images/dep000035/interesting_timelapse/annotated_boxes_species/', x) for x in image_paths]
print(len(image_paths))

In [ ]:
# Open images and convert to a sequence
image_paths = os.listdir('./examples/images/dep000035/interesting_timelapse/annotated_boxes_species/')
image_paths = [os.path.join('./examples/images/dep000035/interesting_timelapse/annotated_boxes_species/', x) for x in image_paths]
images = [Image.open(img) for img in image_paths]

# Save as GIF
gif_path = "./examples/images/dep000035/interesting_timelapse/gifs/species_images.gif"
images[0].save(gif_path, save_all=True, append_images=images[1:], duration=500, loop=0)

# display(Image.open(gif_path))
del images

# Running the Pipeline from Command Line

The entire pipeline can be run from the command line. The commands are shown below for demonstrative purposes using subprocess, but for high throughput analysis we recommend using slurm. There are examples of slurm scripts in the ./slurm_scripts directory: each regional bash file (e.g. `costarica_final.sh`) calls on the sbatch file `array_processor.sh`. 

```python

In [ ]:
import subprocess

chunk_id = 1
batch_size = 20 # runs for 20 images at a time

country='costarica'
region="cri"

credentials_file="./credentials.json"

deployment_id = "dep000035"
output_base_dir=f"./data/{deployment_id}/{country}_test"
json_file = f"./examples/example_keys/interesting_timelapse.json"

os.makedirs(output_base_dir, exist_ok=True)
os.makedirs(f"{output_base_dir}/{deployment_id}", exist_ok=True)

species_model="./models/turing-costarica_v03_resnet50_2024-06-04-16-17_state.pt"
species_labels="./models/03_costarica_data_category_map.json"

In [ ]:
batch_number_padded = f"{chunk_id:04d}"
csv_file = f"{output_base_dir}/{deployment_id}_{batch_number_padded}.csv"
print(f"Results will save to {csv_file}")

In [ ]:
command = [
    "python3", "-m",
    "amber_inferences.cli.perform_inferences",
    "--chunk_id", str(chunk_id),
    "--batch_size", str(batch_size),
    "--json_file", json_file,
    "--output_dir", output_base_dir,
    "--bucket_name", region,
    "--credentials_file", credentials_file,
    "--csv_file", csv_file,
    "--species_model_path", species_model,
    "--species_labels", species_labels,
    "--perform_inference",
    "--remove_image",
    "--box_threshold", "0",
    "--binary_model_path", "./models/moth-nonmoth-effv2b3_20220506_061527_30.pth",
    "--localisation_model_path", "./models/flat_bug_M.pt",
    "--order_model_path", "./models/dhc_best_128.pth",
    "--order_thresholds_path", "./models/thresholdsTestTrain.csv",
    "--skip_processed",
    "--verbose"
]

result = subprocess.run(command, capture_output=True, text=True)

print(result.stdout)

if result.returncode != 0:
    print("STDERR:\n", result.stderr)

# Tracking

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Flatten the embeddings dictionary
def flatten_embeddings_dict(embeddings_dict):
    records = []
    for image_path in embeddings_dict.keys():
        for crop in embeddings_dict[image_path].keys():
            crop_data = embeddings_dict[image_path][crop]
            record = {
                "image_path": image_path,
                "file": crop_data["file"],
                "crop": crop_data["crop"],
                "embedding": crop_data["embedding"],
                "box": crop_data["box"],
                "cropped_image": crop_data["crop_image"]
            }
            records.append(record)
    return records

# Convert to DataFrame
records = flatten_embeddings_dict(embeddings_list)
df = pd.DataFrame(records)

# Extract embeddings and compute pairwise cosine similarity
embeddings = np.stack(df["embedding"].values)

# similarity_matrix = cosine_similarity(embeddings)

# # Create labels for the index/columns of the similarity matrix
# labels = df["file"] + "_" + df["crop"]
# similarity_df = pd.DataFrame(similarity_matrix, index=labels, columns=labels)

In [ ]:
def iou(bb1, bb2) -> float:
    """Finds intersection over union for a bounding box pair"""

    assert bb1[0] < bb1[2], f"Issue in bounding box 1 x_annotation: {bb1[0]} < {bb1[2]}"
    assert bb1[1] < bb1[3], f"Issue in bounding box 1 y_annotation: {bb1[1]} < {bb1[3]}"
    assert bb2[0] < bb2[2], f"Issue in bounding box 2 x_annotation: {bb2[0]} < {bb2[2]}"
    assert bb2[1] < bb2[3], f"Issue in bounding box 2 y_annotation: {bb2[1]} < {bb2[3]}"

    bb1_area = (bb1[2] - bb1[0] + 1) * (bb1[3] - bb1[1] + 1)
    bb2_area = (bb2[2] - bb2[0] + 1) * (bb2[3] - bb2[1] + 1)

    x_min = max(bb1[0], bb2[0])
    x_max = min(bb1[2], bb2[2])
    width = max(0, x_max - x_min + 1)

    y_min = max(bb1[1], bb2[1])
    y_max = min(bb1[3], bb2[3])
    height = max(0, y_max - y_min + 1)

    intersec_area = width * height
    union_area = bb1_area + bb2_area - intersec_area

    iou = np.around(intersec_area / union_area, 2)
    assert 0 <= iou <= 1, "IoU out of bounds"

    return iou


def box_ratio(bb1, bb2) -> float:
    """Finds the ratio of the two bounding boxes"""

    bb1_area = (bb1[2] - bb1[0] + 1) * (bb1[3] - bb1[1] + 1)
    bb2_area = (bb2[2] - bb2[0] + 1) * (bb2[3] - bb2[1] + 1)

    min_area = min(bb1_area, bb2_area)
    max_area = max(bb1_area, bb2_area)

    box_ratio = min_area / max_area
    assert 0 <= box_ratio <= 1, "box ratio out of bounds"

    return box_ratio


def distance_ratio(bb1, bb2, img_diag: float) -> float:
    """finds the distance between the two bounding boxes and normalizes
    by the image diagonal length
    """

    centre_x_bb1 = bb1[0] + (bb1[2] - bb1[0]) / 2
    centre_y_bb1 = bb1[1] + (bb1[3] - bb1[1]) / 2

    centre_x_bb2 = bb2[0] + (bb2[2] - bb2[0]) / 2
    centre_y_bb2 = bb2[1] + (bb2[3] - bb2[1]) / 2

    dist = math.sqrt(
        (centre_x_bb2 - centre_x_bb1) ** 2 + (centre_y_bb2 - centre_y_bb1) ** 2
    )
    max_dist = img_diag

    assert dist <= max_dist, "distance between bounding boxes more than max distance"

    return dist / max_dist


def cosine_similarity(img1_ftrs, img2_ftrs) -> float:
    """
    Finds cosine similarity between a pair of cropped images.

    Uses the feature embeddings array computed from a CNN model.
    """

    cosine_sim = np.dot(img1_ftrs, img2_ftrs) / (
        np.linalg.norm(img1_ftrs) * np.linalg.norm(img2_ftrs)
    )
    assert 0 <= cosine_sim <= 1.000000001, "Cosine similarity score out of bounds"

    return cosine_sim

In [ ]:
from itertools import combinations
import math

results = []
w_cnn =1
w_iou =1
w_box =1
w_dis =1
for crop1, crop2 in combinations(records, 2):
    features1 = crop1["embedding"]
    features2 = crop2["embedding"]
    bb1 = crop1["box"]
    bb2 = crop2["box"]

    # reorder bb1
    bb1 = [bb1['xmin'], bb1['ymin'], bb1['xmax'], bb1['ymax']]
    bb2 = [bb2['xmin'], bb2['ymin'], bb2['xmax'], bb2['ymax']]

    image = Image.open(crop1['image_path']).convert("RGB")
    image_width, image_height = image.size

    diag = math.sqrt(image_width**2 + image_height**2)

    cnn_cost = 1 - cosine_similarity(features1, features2)
    iou_cost = 1 - iou(bb1, bb2)
    box_ratio_cost = 1 - box_ratio(bb1, bb2)
    dist_ratio_cost = distance_ratio(bb1, bb2, diag)

    total_cost = (
        w_cnn * cnn_cost
        + w_iou * iou_cost
        + w_box * box_ratio_cost
        + w_dis * dist_ratio_cost
    )

    results.append([
        crop1['image_path'],
        crop1['crop'],
        crop2['crop'],
        cnn_cost,
        iou_cost,
        box_ratio_cost,
        dist_ratio_cost,
        total_cost
    ])


In [ ]:
results

In [ ]:
columns = [
    "image_path",
    "crop1_id",
    "crop2_id",
    "cnn_cost",
    "iou_cost",
    "box_ratio_cost",
    "dist_ratio_cost",
    "total_cost"
]

results_df = pd.DataFrame(results, columns=columns)

In [ ]:
results_df

In [ ]:
records

In [ ]:
# Step 4: Find pairs with similarity > 0.8 (excluding diagonal)
threshold = 0.9
similar_pairs = []
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        if similarity_matrix[i, j] > threshold:
            similar_pairs.append((i, j, similarity_matrix[i, j]))

In [ ]:
for i, j, sim in similar_pairs:
    crop1, crop2 = df.iloc[i], df.iloc[j]

    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))
    axes[0].imshow(crop1['cropped_image'])
    axes[0].set_title(f"{crop1['file']}\n{crop1['crop']}")
    axes[0].axis("off")

    axes[1].imshow(crop2['cropped_image'])
    axes[1].set_title(f"{crop2['file']}\n{crop2['crop']}")
    axes[1].axis("off")

    plt.suptitle(f"Similarity: {sim:.3f}")
    plt.tight_layout()
    plt.show()